In [1]:
import logging

import probtorch
import torch

import combinators
import filtering
import hmm
import utils

logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

In [2]:
generative = combinators.ParticleTrace(1)

In [3]:
hmm_params = utils.vardict({
    'mu': {
        'loc': torch.arange(5, dtype=torch.float).unsqueeze(0) * 2,
        'scale': torch.ones(1, 5) * 0.25,
    },
    'sigma': {
        'loc': torch.ones(1, 5),
        'scale': torch.ones(1, 5) * 0.25,
    }
})
for k in range(6):
    hmm_params['Pi_%d' % k] = {'concentration': torch.ones(1, 5)}

In [4]:
init_hmm = combinators.Model(f=hmm.init_hmm, hyper=hmm_params)

In [5]:
z0, mu, sigma, pi, pi0 = init_hmm(trace=generative)

In [6]:
hmm_step = combinators.Model(f=hmm.hmm_step)
hmm_run = combinators.Model.sequence(hmm_step, 50, z0, mu, sigma, pi, pi0)

In [7]:
z_last, mu, sigma, pi, pi0 = hmm_run(trace=generative)

In [8]:
generative = generative.squeeze()

In [9]:
num_particles = 100

In [10]:
inference = combinators.ParticleTrace(100)

In [11]:
hmm_params = utils.vardict({
    'mu': {
        'loc': torch.arange(5, dtype=torch.float) * 2,
        'scale': torch.ones(5) * 0.25,
    },
    'sigma': {
        'loc': torch.ones(5),
        'scale': torch.ones(5) * 0.25,
    }
})
for k in range(6):
    hmm_params['Pi_%d' % k] = {'concentration': torch.ones(5)}
    hmm_params['Pi_%d' % k]['concentration'][1] = 1.25
    hmm_params['Pi_%d' % k]['concentration'][2] = 1.5
    hmm_params['Pi_%d' % k]['concentration'][3] = 1.25

In [12]:
init_hmm = combinators.Model(f=hmm.init_hmm, trainable=hmm_params)

In [13]:
z0, mu, sigma, pi, pi0 = init_hmm(trace=inference, guide=generative)

In [14]:
hmm_step = hmm.forward_backward_filter_hmm(mu, sigma, pi, pi0)
hmm_run = combinators.Model.sequence(hmm_step, 50, z0, mu, sigma, pi, pi0)

In [15]:
z_last, mu, sigma, pi, pi0 = hmm_run(trace=inference, guide=generative)

In [16]:
hmm_step.backward_pass(T=50)

In [17]:
hmm_step.smoothed_posterior(T=50)[0][3]

tensor([[ -70.9412,   -2.2684,  -26.3351,   -3.0615,   -0.1629],
        [  -1.8694,   -1.3203,   -0.5519,  -34.3642,   -5.8548],
        [  -0.0325,   -3.4460,  -10.3419,  -63.2763,  -10.2952],
        [ -32.3503,   -1.9944,   -4.7457,   -0.2164,   -2.9992],
        [  -0.8279,   -1.0094,   -1.6219,  -40.4713,   -6.8194],
        [ -10.9286,   -3.0937,   -0.0482,  -17.3179,   -6.3454],
        [  -4.8867,   -1.5197,   -0.2588,  -26.4391,   -6.3750],
        [ -88.2412,   -3.0379,  -38.2772,  -10.2285,   -0.0492],
        [  -5.6874,   -1.0050,   -0.4714,  -23.8744,   -5.0486],
        [ -92.4214,   -3.7039,  -40.9718,  -12.8614,   -0.0249],
        [  -0.0743,   -2.7183,   -5.2435,  -49.4927,   -8.1251],
        [  -0.1317,   -2.1622,   -4.8146,  -49.2066,   -8.3110],
        [  -0.1162,   -2.2115,   -9.1417,  -58.0879,   -9.2340],
        [-101.9741,   -3.6879,  -48.8788,  -18.5026,   -0.0253],
        [ -42.4917,   -4.3703,  -11.2907,   -0.0373,   -3.7334],
        [  -1.3646,   -0.

In [18]:
init_hmm_params = utils.vardict({
    'mu': {
        'loc': torch.rand(5) * 10,
        'scale': torch.ones(5),
    },
    'sigma': {
        'loc': torch.ones(5),
        'scale': torch.ones(5),
    }
})
for k in range(6):
    init_hmm_params['Pi_%d' % k] = {'concentration': torch.ones(5)}

In [19]:
init_hmm = combinators.Model(f=hmm.init_hmm, trainable=init_hmm_params)

In [20]:
def hmm_step_builder(z0, mu, sigma, pi, pi0):
    return hmm.forward_backward_filter_hmm(mu, sigma, pi, pi0)

In [21]:
inference, variational_params = filtering.variational_forward_backward(init_hmm, hmm_step_builder, 500, 50, generative, use_cuda=False, lr=1e-2)

07/30/2018 13:40:46 Variational forward-backward ELBO=-6.92477686e+03 at epoch 1
07/30/2018 13:40:46 Variational forward-backward ELBO=-7.25804297e+03 at epoch 2
07/30/2018 13:40:47 Variational forward-backward ELBO=-7.27300488e+03 at epoch 3
07/30/2018 13:40:47 Variational forward-backward ELBO=-7.16324512e+03 at epoch 4
07/30/2018 13:40:47 Variational forward-backward ELBO=-8.10126221e+03 at epoch 5
07/30/2018 13:40:48 Variational forward-backward ELBO=-7.22321924e+03 at epoch 6
07/30/2018 13:40:48 Variational forward-backward ELBO=-7.16633594e+03 at epoch 7
07/30/2018 13:40:48 Variational forward-backward ELBO=-7.36403711e+03 at epoch 8
07/30/2018 13:40:48 Variational forward-backward ELBO=-7.00788184e+03 at epoch 9
07/30/2018 13:40:49 Variational forward-backward ELBO=-7.44327441e+03 at epoch 10
07/30/2018 13:40:49 Variational forward-backward ELBO=-6.64798926e+03 at epoch 11
07/30/2018 13:40:49 Variational forward-backward ELBO=-7.98010205e+03 at epoch 12
07/30/2018 13:40:50 Varia

In [22]:
variational_params

"{'Pi_0__concentration': 'Parameter containing:\ntensor([ 1.0873, -0.3207,  0.1188,  2.0341,  2.0112], requires_grad=True)', 'Pi_1__concentration': 'Parameter containing:\ntensor([1.1910, 1.1695, 0.8064, 1.5305, 2.0371], requires_grad=True)', 'Pi_2__concentration': 'Parameter containing:\ntensor([ 2.1489, -0.2537,  0.6307,  1.9916,  2.1305], requires_grad=True)', 'Pi_3__concentration': 'Parameter containing:\ntensor([1.3068, 0.4041, 0.7099, 1.7510, 1.9738], requires_grad=True)', 'Pi_4__concentration': 'Parameter containing:\ntensor([-0.2477,  1.9857,  0.2284,  2.0440,  1.9513], requires_grad=True)', 'Pi_5__concentration': 'Parameter containing:\ntensor([0.3114, 2.0009, 0.3660, 0.7430, 2.5252], requires_grad=True)', 'mu__loc': 'Parameter containing:\ntensor([ 4.6014,  8.6577,  7.0228,  1.0094, -0.2890], requires_grad=True)', 'mu__scale': 'Parameter containing:\ntensor([ 0.2978, -0.5490,  0.9250, -0.0756, -0.5277], requires_grad=True)', 'sigma__loc': 'Parameter containing:\ntensor([1.598